# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f371cde2af0>
├── 'a' --> tensor([[ 1.3796, -1.1110, -0.7006],
│                   [ 1.1744,  1.1401,  0.5001]])
└── 'x' --> <FastTreeValue 0x7f371cde2f70>
    └── 'c' --> tensor([[ 0.3564, -0.7416, -0.8685, -0.9736],
                        [-1.7870, -1.5400, -1.0367,  0.6007],
                        [-0.0407,  0.4492, -0.0841,  1.4048]])

In [4]:
t.a

tensor([[ 1.3796, -1.1110, -0.7006],
        [ 1.1744,  1.1401,  0.5001]])

In [5]:
%timeit t.a

63.8 ns ± 0.0689 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f371cde2af0>
├── 'a' --> tensor([[ 1.8801, -0.6568, -0.5673],
│                   [ 0.7254, -0.1056, -1.4913]])
└── 'x' --> <FastTreeValue 0x7f371cde2f70>
    └── 'c' --> tensor([[ 0.3564, -0.7416, -0.8685, -0.9736],
                        [-1.7870, -1.5400, -1.0367,  0.6007],
                        [-0.0407,  0.4492, -0.0841,  1.4048]])

In [7]:
%timeit t.a = new_value

71.7 ns ± 0.115 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.3796, -1.1110, -0.7006],
               [ 1.1744,  1.1401,  0.5001]]),
    x: Batch(
           c: tensor([[ 0.3564, -0.7416, -0.8685, -0.9736],
                      [-1.7870, -1.5400, -1.0367,  0.6007],
                      [-0.0407,  0.4492, -0.0841,  1.4048]]),
       ),
)

In [10]:
b.a

tensor([[ 1.3796, -1.1110, -0.7006],
        [ 1.1744,  1.1401,  0.5001]])

In [11]:
%timeit b.a

60.9 ns ± 0.0313 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.6231,  0.3277,  0.6015],
               [-0.5496,  2.1331, -0.6051]]),
    x: Batch(
           c: tensor([[ 0.3564, -0.7416, -0.8685, -0.9736],
                      [-1.7870, -1.5400, -1.0367,  0.6007],
                      [-0.0407,  0.4492, -0.0841,  1.4048]]),
       ),
)

In [13]:
%timeit b.a = new_value

507 ns ± 0.135 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

813 ns ± 0.161 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11 µs ± 29.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

143 µs ± 310 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 257 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f371ce4e970>
├── 'a' --> tensor([[[ 1.3796, -1.1110, -0.7006],
│                    [ 1.1744,  1.1401,  0.5001]],
│           
│                   [[ 1.3796, -1.1110, -0.7006],
│                    [ 1.1744,  1.1401,  0.5001]],
│           
│                   [[ 1.3796, -1.1110, -0.7006],
│                    [ 1.1744,  1.1401,  0.5001]],
│           
│                   [[ 1.3796, -1.1110, -0.7006],
│                    [ 1.1744,  1.1401,  0.5001]],
│           
│                   [[ 1.3796, -1.1110, -0.7006],
│                    [ 1.1744,  1.1401,  0.5001]],
│           
│                   [[ 1.3796, -1.1110, -0.7006],
│                    [ 1.1744,  1.1401,  0.5001]],
│           
│                   [[ 1.3796, -1.1110, -0.7006],
│                    [ 1.1744,  1.1401,  0.5001]],
│           
│                   [[ 1.3796, -1.1110, -0.7006],
│                    [ 1.1744,  1.1401,  0.5001]]])
└── 'x' --> <FastTreeValue 0x7f371ce4e070>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.6 µs ± 34.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f371cddac70>
├── 'a' --> tensor([[ 1.3796, -1.1110, -0.7006],
│                   [ 1.1744,  1.1401,  0.5001],
│                   [ 1.3796, -1.1110, -0.7006],
│                   [ 1.1744,  1.1401,  0.5001],
│                   [ 1.3796, -1.1110, -0.7006],
│                   [ 1.1744,  1.1401,  0.5001],
│                   [ 1.3796, -1.1110, -0.7006],
│                   [ 1.1744,  1.1401,  0.5001],
│                   [ 1.3796, -1.1110, -0.7006],
│                   [ 1.1744,  1.1401,  0.5001],
│                   [ 1.3796, -1.1110, -0.7006],
│                   [ 1.1744,  1.1401,  0.5001],
│                   [ 1.3796, -1.1110, -0.7006],
│                   [ 1.1744,  1.1401,  0.5001],
│                   [ 1.3796, -1.1110, -0.7006],
│                   [ 1.1744,  1.1401,  0.5001]])
└── 'x' --> <FastTreeValue 0x7f371cdd50d0>
    └── 'c' --> tensor([[ 0.3564, -0.7416, -0.8685, -0.9736],
                        [-1.7870, -1.5400, -1.0367,  0.6007],
                 

In [23]:
%timeit t_cat(trees)

29.7 µs ± 57.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.3 µs ± 50.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.3564, -0.7416, -0.8685, -0.9736],
                       [-1.7870, -1.5400, -1.0367,  0.6007],
                       [-0.0407,  0.4492, -0.0841,  1.4048]],
              
                      [[ 0.3564, -0.7416, -0.8685, -0.9736],
                       [-1.7870, -1.5400, -1.0367,  0.6007],
                       [-0.0407,  0.4492, -0.0841,  1.4048]],
              
                      [[ 0.3564, -0.7416, -0.8685, -0.9736],
                       [-1.7870, -1.5400, -1.0367,  0.6007],
                       [-0.0407,  0.4492, -0.0841,  1.4048]],
              
                      [[ 0.3564, -0.7416, -0.8685, -0.9736],
                       [-1.7870, -1.5400, -1.0367,  0.6007],
                       [-0.0407,  0.4492, -0.0841,  1.4048]],
              
                      [[ 0.3564, -0.7416, -0.8685, -0.9736],
                       [-1.7870, -1.5400, -1.0367,  0.6007],
                       [-0.0407,  0.4492, -0.0841,  1.4048]],

In [26]:
%timeit Batch.stack(batches)

78.2 µs ± 91 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.3564, -0.7416, -0.8685, -0.9736],
                      [-1.7870, -1.5400, -1.0367,  0.6007],
                      [-0.0407,  0.4492, -0.0841,  1.4048],
                      [ 0.3564, -0.7416, -0.8685, -0.9736],
                      [-1.7870, -1.5400, -1.0367,  0.6007],
                      [-0.0407,  0.4492, -0.0841,  1.4048],
                      [ 0.3564, -0.7416, -0.8685, -0.9736],
                      [-1.7870, -1.5400, -1.0367,  0.6007],
                      [-0.0407,  0.4492, -0.0841,  1.4048],
                      [ 0.3564, -0.7416, -0.8685, -0.9736],
                      [-1.7870, -1.5400, -1.0367,  0.6007],
                      [-0.0407,  0.4492, -0.0841,  1.4048],
                      [ 0.3564, -0.7416, -0.8685, -0.9736],
                      [-1.7870, -1.5400, -1.0367,  0.6007],
                      [-0.0407,  0.4492, -0.0841,  1.4048],
                      [ 0.3564, -0.7416, -0.8685, -0.9736],
                   

In [28]:
%timeit Batch.cat(batches)

184 µs ± 330 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

322 µs ± 2.33 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
